Pull Data from FRED and other sources

This notebook is a playground for creating and testing different kinds of models to forecast US Dollar Inflation.  The vast majority of the data I use is extracted from the St. Louis Federal Reserve's FRED API, and some of the stock market data and data on gold prices are from the Wall Street Journal.  This project uses the same data that a different, but related project of mine has already extracted, so the code is copied at the bottom to be re-used for this notebook specifically.

In no particular order, the models that I will build include:
- Various types of linear and non-linear regression, scaling, splitting, and manipulating the data in different ways
- Support Vector Machine
- Random Forest
- Neural Network
- Binning data (understanding the dangers of doing so manually) and using association rule mining
- Creation of models by economic sector using a mix of techniques, and combining them to create an ensemble model 

My ultimate goals are to create precise models using existing techniques, and also to create my own inflation model by combining techniques or by implementing an idea I have to create a model based on vectors (different from SVM).  I will test the performance of all models, and highlight those that perform the best.

In [1]:
# Import Dependencies and API Key

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import json

# Import API key. Git ignore to be used in the future to hide FRED API key, but isn't utilized yet
from api_keys import fred_key

In [20]:
from tensorflow import keras

In [23]:
from sklearn.linear_model import LinearRegression

Model 1: 

In [30]:
m1m2_df

,m1,m1_change,m1_pct_change,m2,m2_change,m2_pct_change,non_m1_components_m2,non_m1_components_m2_change,non_m1_components_m2_pct_change
Date,,,,,,,,,
1959-01-01,138.9,0,0,286.6,0,0,NaN,NaN,NaN
1959-02-01,139.4,0.5,0.359971,287.7,1.1,0.38381,NaN,NaN,NaN
1959-03-01,139.7,0.3,0.215208,289.2,1.5,0.521376,NaN,NaN,NaN
1959-04-01,139.7,0,0,290.1,0.9,0.311203,NaN,NaN,NaN
1959-05-01,140.7,1,0.71582,292.2,2.1,0.723888,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2021-04-01,18915.7,246.3,1.31927,20122.7,209.2,1.05054,NaN,NaN,NaN
2021-05-01,19192.9,277.2,1.46545,20372.4,249.7,1.24089,NaN,NaN,NaN
2021-06-01,19238.2,45.3,0.236025,20387.4,15,0.073629,NaN,NaN,NaN


In [49]:
from sklearn.preprocessing import StandardScaler
m1_change_array = pre_pandemic_m1m2_df['m1_change'].values.reshape(-1,1)
pre_pandemic_m1m2_df['m1_change_scaled']= StandardScaler().fit_transform(m1_change_array)
pre_pandemic_m1m2_df

,Date,m1,m1_change,m1_pct_change,m2,m2_change,m2_pct_change,non_m1_components_m2,non_m1_components_m2_change,non_m1_components_m2_pct_change,m1_change_scaled
152,1971-09-01,226.5,0.9,0.398936,692.5,7,1.02115,NaN,NaN,NaN,-0.386106
153,1971-10-01,227.2,0.7,0.309051,698.4,5.9,0.851986,NaN,NaN,NaN,-0.399813
154,1971-11-01,227.8,0.6,0.264085,704.6,6.2,0.887743,NaN,NaN,NaN,-0.406666
155,1971-12-01,228.3,0.5,0.219491,710.3,5.7,0.80897,NaN,NaN,NaN,-0.413519
156,1972-01-01,230.1,1.8,0.788436,717.7,7.4,1.04181,NaN,NaN,NaN,-0.324428
...,...,...,...,...,...,...,...,...,...,...,...
3005,2020-01-06,NaN,NaN,NaN,NaN,NaN,NaN,11399.8,20.9,0.183673,NaN
3006,2020-01-13,NaN,NaN,NaN,NaN,NaN,NaN,11434.2,34.4,0.30176,NaN
3007,2020-01-20,NaN,NaN,NaN,NaN,NaN,NaN,11456.5,22.3,0.195029,NaN
3008,2020-01-27,NaN,NaN,NaN,NaN,NaN,NaN,11493.6,37.1,0.323834,NaN


In [147]:
m2_df = m1m2_df[['m2','m2_change','m2_pct_change']].reset_index()
government_expenditures_df = government_quarterly_df[['government_expenditures','government_expenditures_change','government_expenditures_pct_change']].reset_index()
net_exports_df = foreign_trade_month_quarter_df[['net_exports','net_exports_change','net_exports_pct_change']].reset_index()
inflation_expectation_df = cpi_monthly_df[['inflation_expectation','inflation_expectation_change','inflation_expectation_pct_change']].reset_index()
gdp_df = gdp_quarterly_df[['gdp','gdp_change','gdp_pct_change']].reset_index()
reserve_balances_df = banks_week_month_df[['reserve_balances','reserve_balances_change','reserve_balances_pct_change']].reset_index()
unemployment_df = consumer_monthly_df[['unemployment','unemployment_change','unemployment_pct_change']].reset_index()
real_output_hour_df = consumers_quarterly_df[['real_output_hour','real_output_hour_change','real_output_hour_pct_change']].reset_index()
ppi_all_commodities_df = ppi_monthly_df[['ppi_all_commodities','ppi_all_commodities_change','ppi_all_commodities_pct_change']].reset_index()
cpi_df = cpi_monthly_df[['cpi','cpi_change','cpi_pct_change']].reset_index()
basic_neural_predictor_df_list = [government_expenditures_df,net_exports_df,inflation_expectation_df,gdp_df,reserve_balances_df,\
                                 unemployment_df,real_output_hour_df,ppi_all_commodities_df,cpi_df]
non_df_list = ['government_expenditures','net_exports','inflation_expectation','gdp','reserve_balances','unemployment','real_output_hour',\
              'ppi_all_commodities','cpi']

In [145]:
m2_df.iloc[:,1]

0         286.6
1         287.7
2         289.2
3         290.1
4         292.2
         ...   
3074    20122.7
3075    20372.4
3076    20387.4
3077    20534.3
3078      20797
Name: m2, Length: 3079, dtype: object

In [148]:
##### Need to write for loop to scale all data############
m2_change_array = m2_df['m2_change'].values.reshape(-1,1)
m2_df['m2_scaled_change']= StandardScaler().fit_transform(m2_change_array)
m2_change_array = m2_df['m2_pct_change'].values.reshape(-1,1)
m2_df['m2_scaled_pct_change']= StandardScaler().fit_transform(m2_change_array)
scaled_change_df = m2_df
# basic_neural_all_df = m2_df
for i in range(len(basic_neural_predictor_df_list)):
    change_array = basic_neural_predictor_df_list[i].iloc[:,2].values.reshape(-1,1)
    pct_change_array = basic_neural_predictor_df_list[i].iloc[:,3].values.reshape(-1,1)
    basic_neural_predictor_df_list[i][f'{non_df_list[i]}_scaled_change'] = StandardScaler().fit_transform(change_array)
    basic_neural_predictor_df_list[i][f'{non_df_list[i]}_scaled_pct_change'] = StandardScaler().fit_transform(pct_change_array)
    scaled_change_df = scaled_change_df.merge(basic_neural_predictor_df_list[i], how="inner", on="Date")
scaled_change_df
    
# m1_change_array = pre_pandemic_m1m2_df['m1_change'].values.reshape(-1,1)
# pre_pandemic_m1m2_df['m1_change_scaled']= StandardScaler().fit_transform(m1_change_array)
# pre_pandemic_m1m2_df

,Date,m2,m2_change,m2_pct_change,m2_scaled_change,m2_scaled_pct_change,government_expenditures,government_expenditures_change,government_expenditures_pct_change,government_expenditures_scaled_change,...,ppi_all_commodities,ppi_all_commodities_change,ppi_all_commodities_pct_change,ppi_all_commodities_scaled_change,ppi_all_commodities_scaled_pct_change,cpi,cpi_change,cpi_pct_change,cpi_scaled_change,cpi_scaled_pct_change
0,1959-01-01,286.6,0,0,-0.434443,-1.258333,NaN,NaN,NaN,NaN,...,31.7,0.1,0.316456,-0.065949,0.063486,29.01,0.04,0.138074,-0.579792,-0.427306
1,1959-04-01,290.1,0.9,0.311203,-0.420108,-0.574183,NaN,NaN,NaN,NaN,...,31.8,0.1,0.315457,-0.065949,0.062706,28.98,0.01,0.0345185,-0.652047,-0.729013
2,1959-07-01,295.2,1.1,0.374022,-0.416922,-0.436081,NaN,NaN,NaN,NaN,...,31.7,0,0,-0.161079,-0.183832,29.15,0.04,0.13741,-0.579792,-0.429240
3,1959-10-01,296.5,-0.2,-0.0674082,-0.437629,-1.406524,NaN,NaN,NaN,NaN,...,31.6,-0.1,-0.315457,-0.256209,-0.430370,29.35,0.1,0.34188,-0.435282,0.166481
4,1960-01-01,298.2,0.4,0.134318,-0.428072,-0.963047,144.233,0,0,-0.121240,...,31.6,0.1,0.31746,-0.065949,0.064272,29.37,-0.04,-0.136008,-0.772472,-1.225839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,2020-04-01,17039.1,1027.7,6.41855,15.935348,12.852238,10910.4,3439.67,46.0418,11.150340,...,185.5,-7.6,-3.93578,-7.390949,-3.259746,256.192,-1.797,-0.696541,-5.004202,-2.858943
246,2020-07-01,18316.6,140.9,0.775211,1.809892,0.445894,9706.16,-1204.26,-11.0377,-4.067515,...,193,1.8,0.941423,1.551259,0.551914,258.604,1.322,0.513833,2.507902,0.667462
247,2020-10-01,18747.9,146.3,0.786491,1.895906,0.470693,8471.92,-1234.24,-12.716,-4.165767,...,196.5,1,0.511509,0.790220,0.215925,260.462,0.313,0.120316,0.077728,-0.479044
248,2021-01-01,19393.1,263.6,1.37798,3.764328,1.771016,10790.8,2318.91,27.3717,7.477673,...,204.8,4.3,2.14464,3.929506,1.492257,262.231,0.671,0.256538,0.939970,-0.082163


In [149]:
# basic_neural_all_df = basic_neural_all_df.set_index("Date")
scaled_change_df = scaled_change_df.loc[scaled_change_df['Date']<='2020-02-01',:]
scaled_change_df = scaled_change_df.loc[scaled_change_df['Date']>='1971-09-01',:]
scaled_change_df = scaled_change_df.reset_index()
scaled_change_df

,index,Date,m2,m2_change,m2_pct_change,m2_scaled_change,m2_scaled_pct_change,government_expenditures,government_expenditures_change,government_expenditures_pct_change,...,ppi_all_commodities,ppi_all_commodities_change,ppi_all_commodities_pct_change,ppi_all_commodities_scaled_change,ppi_all_commodities_scaled_pct_change,cpi,cpi_change,cpi_pct_change,cpi_scaled_change,cpi_scaled_pct_change
0,51,1971-10-01,698.4,5.9,0.851986,-0.340465,0.614675,383.306,6.174,1.63709,...,38.3,0,0,-0.161079,-0.183832,40.9,0.1,0.245098,-0.435282,-0.115493
1,52,1972-01-01,717.7,7.4,1.04181,-0.316572,1.031993,399.428,16.122,4.20604,...,38.8,0.2,0.518135,0.029181,0.221104,41.2,0.1,0.243309,-0.435282,-0.120705
2,53,1972-04-01,738.4,4.9,0.66803,-0.356393,0.210267,403.929,4.501,1.12686,...,39.3,0.1,0.255102,-0.065949,0.015537,41.5,0.1,0.241546,-0.435282,-0.125842
3,54,1972-07-01,759.5,9.8,1.30719,-0.278343,1.615398,404.908,0.979,0.242369,...,40,0.3,0.755668,0.124311,0.406741,41.8,0.1,0.239808,-0.435282,-0.130905
4,55,1972-10-01,786.9,8.6,1.10497,-0.297458,1.170843,419.285,14.377,3.55068,...,40.1,-0.1,-0.248756,-0.256209,-0.378241,42.2,0.1,0.23753,-0.435282,-0.137543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,240,2019-01-01,14443.3,69.4,0.48282,0.670999,-0.196901,7165.64,116.589,1.65397,...,199.1,-1.9,-0.945274,-1.968546,-0.922587,252.441,-0.052,-0.0205946,-0.801374,-0.889584
190,241,2019-04-01,14568.9,35.4,0.243575,0.129428,-0.722857,7243.44,77.8,1.08574,...,202.1,1.3,0.64741,1.075610,0.322136,255.326,1.179,0.463905,2.163487,0.521997
191,242,2019-07-01,14857.9,70.9,0.479475,0.694892,-0.204253,7319.77,76.329,1.05377,...,200.7,0.4,0.1997,0.219441,-0.027761,255.925,0.502,0.196537,0.532934,-0.256975
192,243,2019-10-01,15153.1,126.5,0.84184,1.580521,0.592372,7418.35,98.588,1.34687,...,198.6,0.2,0.100806,0.029181,-0.105049,257.387,0.855,0.333292,1.383133,0.141458


In [136]:
# def clean_dataset(df):
#     assert isinstance(df, pd.DataFrame)
#     df.dropna(inplace=True)
#     indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
#     return df[indices_to_keep].astype(np.float64)

In [150]:
# column_list_basic_neural = list(basic_neural_all_df)
# for each_column in column_list_basic_neural:
#     print('The max of ', each_column, round(basic_neural_all_df[each_column].max(),3))
#     print('The min of ',each_column,round(basic_neural_all_df[each_column].min(),3))

In [151]:
# cpi_df = cpi_monthly_df[['cpi','cpi_change','cpi_pct_change']]
# cpi_df = cpi_df.reset_index()
# cpi_df = cpi_df.loc[cpi_df['Date']<='2020-02-01',:]
# cpi_df = cpi_df.loc[cpi_df['Date']>='1971-09-01',:]
# cpi_df = cpi_df.set_index('Date')
# cpi_df

In [158]:
X = scaled_change_df[['m2_scaled_change','government_expenditures_scaled_change','net_exports_scaled_change',\
                         'gdp_scaled_change','reserve_balances_scaled_change','unemployment_scaled_change','real_output_hour_scaled_change',\
                         'ppi_all_commodities_scaled_change']]
y = scaled_change_df[['cpi_scaled_change']]

In [159]:
print(X.shape)
print(y.shape)

(194, 8)
(194, 1)


In [157]:
X

,m2_scaled_change,government_expenditures_scaled_change,net_exports_scaled_change,inflation_expectation_scaled_change,gdp_scaled_change,reserve_balances_scaled_change,unemployment_scaled_change,real_output_hour_scaled_change,ppi_all_commodities_scaled_change
0,-0.340465,-0.101008,0.030604,NaN,-0.353175,-0.101032,-0.474898,-1.412485,-0.161079
1,-0.316572,-0.068409,0.050371,NaN,-0.197635,-0.071535,-0.474898,0.822165,0.029181
2,-0.356393,-0.106490,0.082828,NaN,-0.223205,-0.088127,-0.239312,1.225333,-0.065949
3,-0.278343,-0.118032,0.168312,NaN,-0.288156,-0.089971,-0.239312,-0.059134,0.124311
4,-0.297458,-0.074127,0.093806,NaN,-0.208723,-0.086283,0.231859,0.206269,-0.256209
...,...,...,...,...,...,...,...,...,...
189,0.670999,0.260815,1.368151,0.002689,0.633451,-1.070732,0.231859,1.203048,-1.968546
190,0.129428,0.133706,-0.834848,0.002689,1.191862,-1.981440,-0.239312,1.679152,1.075610
191,0.694892,0.128885,0.777057,-0.232120,0.787801,-0.519515,-0.003727,-0.182718,0.219441
192,1.580521,0.201827,3.084614,-0.701737,0.640080,0.678785,0.231859,-0.292121,0.029181


In [155]:
from sklearn.neural_network import MLPRegressor
basic_neural = MLPRegressor(hidden_layer_sizes=(512))

In [156]:
basic_neural_model = basic_neural.fit(X,y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

Extract Economic Data

In [2]:
#### Dictionary containing the FRED code to pull data from the API as the keys, and the name of the corresponsing main series
#### as named in my database as the values.

data_extract_dict = {'M1SL':'m1',
                      'M2SL':'m2',
                      'NONM1':'non_m1_components_m2',
                      'M1V':'m1v',
                      'M2V':'m2v',
                      'BUSLOANS':'commercial_industrial_loans',
                      'FEDFUNDS':'fed_funds_rate',
                      'DPSACBW027SBOG':'commercial_bank_deposits',
                      'TLAACBW027SBOG':'commercial_bank_assets',
                      'TOTRESNS':'reserve_balances',
                      'TOTBKCR':'commercial_bank_credit',
                      'MORTGAGE30US':'_30yr_fixed_rate_mortgage',
                      'CONSUMER':'consumer_loans_com_banks',
                      'CASACBW027SBOG':'commercial_bank_cash_assets',
                      'POPTHM':'pop',
                      'PCEPI':'pce_index',
                      'UNRATE':'unemployment',
                      'PSAVERT':'personal_savings_rate',
                      'CES0500000003':'average_hourly_wage',
                      'PMSAVE':'personal_savings',
                      'CUUR0000SETA01':'cpi_vehicles',
                      'APU0000708111':'cpi_eggs',
                      'CPIAPPSL':'cpi_apparel_cities',
                      'CPIHOSNS':'cpi_housing_cities',
                      'PCEDGC96':'real_pce_durable_goods',
                      'CPITRNSL':'cpi_urban_transportation',
                      'PCE':'pce',
                      'CIVPART':'labor_participation_rate',
                      'PCEC96':'real_pce',
                      'PCEDG':'pce_durable_goods',
                      'JTSJOL':'job_openings_nonfarm',
                      'PCEND':'pce_nondurable_goods',
                      'DSPIC96':'real_disposable_personal_income',
                      'ECOMPCTSA':'ecommerce_pct_of_totalsales',
                      'MSPUS':'median_house_sale_price',
                      'HDTGPDUSQ163N':'house_debt_gdp_ratio',
                      'OPHNFB':'real_output_hour',
                      'RRVRUSQ156N':'rental_vacancy_rate',
                      'DRSFRMACBS':'mortgage_delinquency',
                      'TDSP':'household_debt_service_pmtpctgdp',
                      'RHORUSQ156N':'homeownership_rate',
                      'DRCCLACBS':'creditcard_delinquency_rate',
                      'WFRBST01134':'wealth_share_top1pct',
                      'GPSAVE':'gross_private_saving',
                      'QUSR628BIS':'real_residential_property_price',
                      'WFRBLB50107':'bottom_50pct_net_worth',
                      'NCBCMDPMVCE':'debt_as_pct_corporate_equities',
                      'WFRBLT01026':'wealth_total_top1pct',
                      'DRCLACBS':'consumer_loan_delinquency_rate',
                      'CPIAUCSL':'cpi',
                      'MICH':'inflation_expectation',
                      'CPILFESL':'cpi_core',
                      'CPIMEDSL':'cpi_medical',
                      'CUUR0000SA0R':'cpi_urban',
                      'CPIFABSL':'cpi_food_bev',
                      'STLFSI2':'financial_stress',
                      'WALCL':'fed_assets',
                      'TREAST':'fed_res_held_treasuries',
                      'WTREGEN':'fed_liabilities_non_reserve_deposits',
                      'RESPPANWW':'total_fed_assets',
                      'BOPGSTB':'net_trade',
                      'IMPGSC1':'real_imports',
                      'IMPGS':'imports_goods_services',
                      'INDCPIALLMINMEI':'cpi_india',
                      'IMPCH':'imports_from_china',
                      'IR':'all_commodities_import_price_index',
                      'GDP':'gdp',
                      'A939RC0Q052SBEA':'nom_gdpcap',
                      'GDPC1':'real_gdp',
                      'A939RX0Q048SBEA':'real_gdpcap',
                      'GDPDEF':'gdp_deflator',
                      'GFDEBTN':'federal_debt',
                      'GFDEGDQ188S':'debt_pct_gdp',
                      'W068RCQ027SBEA':'government_expenditures',
                      'FYGFDPUN':'federal_debt_held_by_public',
                      'FDHBFRBN':'fr_held_debt',
                      'B087RC1Q027SBEA':'government_transfer_payments',
                      'M318501Q027NBEA':'federal_surplus_deficit',
                      'B075RC1Q027SBEA':'corporate_income_tax_receipts',
                      'TTLCONS':'construction_spending',
                      'HOUST':'housing_starts',
                      'GPDIC1':'real_gross_domestic_private_investment',
                      'FYFSD':'deficit_surplus',
                      'MEHOINUSA672N':'real_median_house_income',
                      'FPCPITOTLZGUSA':'inflation_consumer_price',
                      'USEPUINDXD':'economic_uncertainty',
                      'PPIACO':'ppi_all_commodities',
                      'WPU0911':'ppi_wood_pulp',
                      'WPU101707':'ppi_metals',
                      'PCU325211325211':'ppi_plastics_resins',
                      'WPU101':'ppi_iron_steel',
                      'PWHEAMTUSDQ':'global_wheat_price',
                      'WPU10170502':'ppi_steel_wire',
                      'PCU484121484121':'ppi_freight',
                      'PALUMUSDM':'global_aluminum_price',
                      'PCU44414441':'ppi_building_materials',
                      'WPU0811':'ppi_wood_lumber',
                      'PMAIZMTUSDM':'global_corn_price',
                      'PIORECRUSDM':'global_iron_price',
                      'PRUBBUSDM':'global_rubber_price',
                      'WPU081':'ppi_lumber',
                      'PCU32733273':'ppi_cement_concrete',
                      'PCU33443344':'ppi_semiconductors_electronics',
                      'UMCSENT':'consumer_sentiment',
                      'CP':'corporate_profits_after_tax',
                      'PCESV':'pce_services',
                      'CUUR0000SEHA':'cpi_primary_rent',
                      'WSHOMCB':'fed_mbs',
                      'NETEXP':'net_exports',
                      'A019RE1A156NBEA':'net_exports_pctofgdp',
                      'GNP':'gnp',
                      'GPDI':'gross_domestic_private_investment',
                      'DCOILWTICO':'price_per_barrel',
                      'T10YIE':'_10_year_breakeven_inflation',
                      'T5YIFR':'inf_expectation_5yr',
                      'PCOPPUSDM':'copper_price',
                      'PCUOMFGOMFG':'ppi_manufacturing',
                      'DDDM01USA156NWDB':'stock_market_cap'}

In [7]:
#### Defines main function for extracting and transforming data.  This function is used on series with no known calculation errors
#### The function extracts raw data from the API in JSON format, pulls dates and observed valuse from the JSON, storing them in 
#### lists, runs calculations on the stored values for the _change and _pct_change columns for each series, storing those calculated
#### values in additional lists, and appends all three lists that correspond with each series to the all_data dictionary.  Error
#### handling is included with the try/except language so that the entire function does not stop if there is an unexpected error
#### when extracting or transforming a single series.

all_data = {}
def fred_extract(series_dict):
    for key, value in series_dict.items():
        try:
            data = requests.get(f'https://api.stlouisfed.org/fred/series/observations?series_id={key}&api_key={fred_key}&file_type=json')
            series_json = data.json()
            series_json_obs=series_json['observations']
            series_dates = []
            series_values = []
            series_change_values = [0]
            series_pct_change_values = [0]
            change_index = 1
            for each_item in range(len(series_json_obs)):
                item_date=series_json_obs[each_item]['date']
                item_value=series_json_obs[each_item]['value']
                if item_value != ".":
                    series_dates.append(series_json_obs[each_item]['date'])
                    series_values.append(float(series_json_obs[each_item]['value']))
        except:
            print(f'Error extracting {key}')
        for each_value in range(len(series_values)-1):
            try:
                if (series_values[change_index] > series_values[each_value]):
                    if (series_values[each_value] > 0):
                        change = series_values[change_index]-series_values[each_value]
                        pct_change = (change/series_values[each_value])*100
                        series_change_values.append(change)
                        series_pct_change_values.append(pct_change)
                    elif (series_values[each_value] < 0):
                        change = series_values[change_index]-series_values[each_value]
                        pct_change = abs(change/series_values[each_value])*100
                        series_change_values.append(change)
                        series_pct_change_values.append(pct_change)
                    elif (series_values[each_value] == 0):
                        change = series_values[change_index]
                        pct_change = 100
                        series_change_values.append(change)
                        series_pct_change_values.append(pct_change)
                elif (series_values[change_index] < series_values[each_value]):
                    if (series_values[each_value] > 0):
                        change = series_values[change_index]-series_values[each_value]
                        pct_change = (change/series_values[each_value])*100
                        series_change_values.append(change)
                        series_pct_change_values.append(pct_change)
                    elif (series_values[each_value] < 0):
                        change = series_values[change_index]-series_values[each_value]
                        pct_change = (abs(change)/series_values[each_value])*100
                        series_change_values.append(change)
                        series_pct_change_values.append(pct_change)
                    elif (series_values[each_value] == 0):
                        change = series_values[change_index]
                        pct_change = -100
                        series_change_values.append(change)
                        series_pct_change_values.append(pct_change)
                elif (series_values[change_index] == series_values[each_value]):
                    change = 0
                    pct_change = 0
                    series_change_values.append(change)
                    series_pct_change_values.append(pct_change)
                elif (series_values[change_index] < series_values[each_value]):
                    if (series_values[each_value] == 0):
                        change = series_values[change_index]
                        pct_change = -100
                        series_change_values.append(change)
                        series_pct_change_values.append(pct_change)
                change_index = change_index + 1
                all_data[value]=[series_dates,series_values,series_change_values,series_pct_change_values]
            except:
                print(f'Error running calculations on {value}') 
            

In [26]:
#### Creates 14 Pandas DataFrames that correspond with the first main series that appear in each of my database tables.
#### The loop in the cell below creates individual DataFrames for each remaining series and merges them with these 14

banks_week_month_df = pd.DataFrame(all_data['commercial_industrial_loans']).transpose().rename(columns={0:"Date",1:"commercial_industrial_loans",2:"commercial_industrial_loans_change",3:"commercial_industrial_loans_pct_change"}).set_index(['Date'])
m1m2_df = pd.DataFrame(all_data['m1']).transpose().rename(columns={0:"Date",1:"m1",2:"m1_change",3:"m1_pct_change"}).set_index(['Date'])
consumer_monthly_df = pd.DataFrame(all_data['pop']).transpose().rename(columns={0:"Date",1:"pop",2:"pop_change",3:"pop_pct_change"}).set_index(['Date'])
consumers_quarterly_df = pd.DataFrame(all_data['ecommerce_pct_of_totalsales']).transpose().rename(columns={0:"Date",1:"ecommerce_pct_of_totalsales",2:"ecommerce_pct_of_totalsales_change",3:"ecommerce_pct_of_totalsales_pct_change"}).set_index(['Date'])
cpi_monthly_df = pd.DataFrame(all_data['cpi']).transpose().rename(columns={0:"Date",1:"cpi",2:"cpi_change",3:"cpi_pct_change"}).set_index(['Date'])
federal_reserve_weekly_df = pd.DataFrame(all_data['financial_stress']).transpose().rename(columns={0:"Date",1:"financial_stress",2:"financial_stress_change",3:"financial_stress_pct_change"}).set_index(['Date'])
foreign_trade_month_quarter_df = pd.DataFrame(all_data['net_trade']).transpose().rename(columns={0:"Date",1:"net_trade",2:"net_trade_change",3:"net_trade_pct_change"}).set_index(['Date'])
gdp_quarterly_df = pd.DataFrame(all_data['gdp']).transpose().rename(columns={0:"Date",1:"gdp",2:"gdp_change",3:"gdp_pct_change"}).set_index(['Date'])
government_quarterly_df = pd.DataFrame(all_data['federal_debt']).transpose().rename(columns={0:"Date",1:"federal_debt",2:"federal_debt_change",3:"federal_debt_pct_change"}).set_index(['Date'])
investment_month_quarter_df = pd.DataFrame(all_data['construction_spending']).transpose().rename(columns={0:"Date",1:"construction_spending",2:"construction_spending_change",3:"construction_spending_pct_change"}).set_index(['Date'])
misc_annual_df = pd.DataFrame(all_data['deficit_surplus']).transpose().rename(columns={0:"Date",1:"deficit_surplus",2:"deficit_surplus_change",3:"deficit_surplus_pct_change"}).set_index(['Date'])
misc_daily_df = pd.DataFrame(all_data['inf_expectation_5yr']).transpose().rename(columns={0:"Date",1:"inf_expectation_5yr",2:"inf_expectation_5yr_change",3:"inf_expectation_5yr_pct_change"}).set_index(['Date'])
ppi_monthly_df = pd.DataFrame(all_data['ppi_manufacturing']).transpose().rename(columns={0:"Date",1:"ppi_manufacturing",2:"ppi_manufacturing_change",3:"ppi_manufacturing_pct_change"}).set_index(['Date'])
velocity_df = pd.DataFrame(all_data['m1v']).transpose().rename(columns={0:"Date",1:"m1v",2:"m1v_change",3:"m1v_pct_change"}).set_index(['Date'])

In [27]:
#### Lists of the main series columns for each table.  Additionally, for each of these series, "_change" and "pct_change" are 
#### calculated in the extraction and transformation functions.  These lists serve to pull specific series from the all_data 
#### dictionary where all data extracted from the FRED API is stored.

m1m2_column_list = ['m2','non_m1_components_m2']
velocity_column_list = ['m2v']
banks_week_month_column_list = ['fed_funds_rate','commercial_bank_deposits','commercial_bank_assets',\
                               'reserve_balances','commercial_bank_credit','_30yr_fixed_rate_mortgage','consumer_loans_com_banks',\
                               'commercial_bank_cash_assets']
consumer_monthly_column_list = ['pce_index','unemployment','personal_savings_rate','average_hourly_wage','personal_savings',\
                               'consumer_sentiment','cpi_vehicles','cpi_eggs','cpi_apparel_cities','cpi_housing_cities',\
                               'real_pce_durable_goods','cpi_urban_transportation','pce','labor_participation_rate','real_pce',\
                               'pce_durable_goods','job_openings_nonfarm','pce_nondurable_goods','real_disposable_personal_income']
consumers_quarterly_column_list = ['median_house_sale_price','house_debt_gdp_ratio','real_output_hour',\
                                  'corporate_profits_after_tax','pce_services','rental_vacancy_rate','mortgage_delinquency',\
                                  'household_debt_service_pmtpctgdp','homeownership_rate','creditcard_delinquency_rate','wealth_share_top1pct',\
                                  'gross_private_saving','real_residential_property_price','bottom_50pct_net_worth',\
                                  'debt_as_pct_corporate_equities','wealth_total_top1pct','consumer_loan_delinquency_rate']
cpi_monthly_column_list = ['inflation_expectation','cpi_core','cpi_medical','cpi_urban','cpi_primary_rent','cpi_food_bev']
federal_reserve_weekly_column_list = ['fed_assets','fed_res_held_treasuries','fed_liabilities_non_reserve_deposits',\
                                     'fed_mbs','total_fed_assets']
foreign_trade_month_quarter_column_list = ['net_exports','net_exports_pctofgdp','real_imports','imports_goods_services',\
                                          'cpi_india','imports_from_china','all_commodities_import_price_index']
gdp_quarterly_column_list = ['nom_gdpcap','real_gdp','real_gdpcap','gdp_deflator','gnp']
government_quarterly_column_list = ['debt_pct_gdp','government_expenditures','federal_debt_held_by_public','fr_held_debt',\
                                   'government_transfer_payments','federal_surplus_deficit','corporate_income_tax_receipts']
investment_month_quarter_column_list = ['housing_starts','real_gross_domestic_private_investment',\
                                       'gross_domestic_private_investment']
misc_annual_column_list = ['stock_market_cap','real_median_house_income','inflation_consumer_price']
misc_daily_column_list = ['price_per_barrel','economic_uncertainty','_10_year_breakeven_inflation']
ppi_monthly_column_list = ['ppi_all_commodities','ppi_wood_pulp','ppi_metals','copper_price','ppi_plastics_resins',\
                          'ppi_iron_steel','global_wheat_price','ppi_steel_wire','ppi_freight','global_aluminum_price',\
                          'ppi_building_materials','ppi_wood_lumber','global_corn_price','global_iron_price','global_rubber_price',\
                          'ppi_lumber','ppi_cement_concrete','ppi_semiconductors_electronics']
#stocks_gold_daily_column_list = ['djia_close','nasdaq_close','sp500_close','gold_price']
# all_table_column_dict = {m1m2_df:m1m2_column_list,velocity_df:velocity_column_list,banks_week_month_df:banks_week_month_column_list,\
#                          consumer_monthly_df:consumer_monthly_column_list,consumers_quarterly_df:consumers_quarterly_column_list,\
#                          cpi_monthly_df:cpi_monthly_column_list,federal_reserve_weekly_df:federal_reserve_weekly_column_list,\
#                          foreign_trade_month_quarter_df:foreign_trade_month_quarter_column_list,gdp_quarterly_df:gdp_quarterly_column_list,\
#                          government_quarterly_df:government_quarterly_column_list,investment_month_quarter_df:investment_month_quarter_column_list,\
#                          misc_annual_df:misc_annual_column_list,misc_daily_df:misc_daily_column_list,ppi_monthly_df:ppi_monthly_column_list}
# all_table_column_list = [m1m2_column_list, velocity_column_list]
# all_table_df_list = [m1m2_df,velocity_df]

In [28]:
for column in m1m2_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    m1m2_df = m1m2_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in velocity_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    velocity_df = velocity_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in banks_week_month_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    banks_week_month_df = banks_week_month_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in consumer_monthly_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    consumer_monthly_df = consumer_monthly_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in consumers_quarterly_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    consumers_quarterly_df = consumers_quarterly_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in cpi_monthly_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    cpi_monthly_df = cpi_monthly_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in federal_reserve_weekly_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    federal_reserve_weekly_df = federal_reserve_weekly_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in foreign_trade_month_quarter_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    foreign_trade_month_quarter_df = foreign_trade_month_quarter_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in gdp_quarterly_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    gdp_quarterly_df = gdp_quarterly_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in government_quarterly_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    government_quarterly_df = government_quarterly_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in investment_month_quarter_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    investment_month_quarter_df = investment_month_quarter_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in misc_annual_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    misc_annual_df = misc_annual_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in misc_daily_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    misc_daily_df = misc_daily_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in ppi_monthly_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    ppi_monthly_df = ppi_monthly_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])

In [29]:
#### Import data manually pulled from WSJ (to be automated with web scraping at a later date), run change and pct_change calculations
#### organize into individual DataFrames, merge DataFrames.  Gold must be cleaned separately before merging
djia_path = os.path.join("../Manual Downloads/djia.csv")
nasdaq_path = os.path.join("../Manual Downloads/nasdaq.csv")
gold_path=os.path.join("../Manual Downloads/gold.csv")
sp500_path=os.path.join("../Manual Downloads/sp500.csv")
djia_data = pd.read_csv(djia_path)
nasdaq_data = pd.read_csv(nasdaq_path)
gold_data=pd.read_csv(gold_path)
sp500_data=pd.read_csv(sp500_path)
sp500_df=sp500_data.rename(columns={' Close':'sp500_close'}).drop(columns=[' Open',' High',' Low']).set_index('Date')
sp500_df=sp500_df[::-1]
sp500_values = sp500_df['sp500_close'].tolist()
sp500_close_change = [0]
sp500_close_pct_change =[0]
change_index = 1
for i in range(len(sp500_values)-1):
    change = sp500_values[change_index] - sp500_values[i]
    pct_change = (change/sp500_values[i])*100
    sp500_close_change.append(change)
    sp500_close_pct_change.append(pct_change)
    change_index=change_index + 1
sp500_df['sp500_close_change']=sp500_close_change
sp500_df['sp500_close_pct_change']=sp500_close_pct_change
djia_df=djia_data.rename(columns={' Close':'djia_close'}).drop(columns=[' Open',' High',' Low']).set_index('Date')
djia_df=djia_df[::-1]
djia_values = djia_df['djia_close'].tolist()
djia_close_change = [0]
djia_close_pct_change =[0]
change_index = 1
for j in range(len(djia_values)-1):
    change = djia_values[change_index] - djia_values[j]
    pct_change = (change/djia_values[i])*100
    djia_close_change.append(change)
    djia_close_pct_change.append(pct_change)
    change_index=change_index + 1
djia_df['djia_close_change']=djia_close_change
djia_df['djia_close_pct_change']=djia_close_pct_change
nasdaq_df=nasdaq_data.rename(columns={' Close':'nasdaq_close'}).drop(columns=[' Open',' High',' Low']).set_index('Date')
nasdaq_df=nasdaq_df[::-1]
nasdaq_values = nasdaq_df['nasdaq_close'].tolist()
nasdaq_close_change = [0]
nasdaq_close_pct_change =[0]
change_index = 1
for q in range(len(nasdaq_values)-1):
    change = nasdaq_values[change_index] - nasdaq_values[q]
    pct_change = (change/nasdaq_values[i])*100
    nasdaq_close_change.append(change)
    nasdaq_close_pct_change.append(pct_change)
    change_index=change_index + 1
nasdaq_df['nasdaq_close_change']=nasdaq_close_change
nasdaq_df['nasdaq_close_pct_change']=nasdaq_close_pct_change
gold_df=gold_data.rename(columns={'DATE':'Date','GOLDPMGBD228NLBM':'gold_price'}).set_index('Date')
gold_df=gold_df.loc[gold_df.loc[:,'gold_price']!='.',:]
gold_values = gold_df['gold_price'].tolist()
gold_price_change = [0]
gold_price_pct_change =[0]
change_index = 1
for g in range(len(gold_values)-1):
    change = float(gold_values[change_index]) - float(gold_values[g])
    pct_change = (change/float(gold_values[g]))*100
    gold_price_change.append(change)
    gold_price_pct_change.append(pct_change)
    change_index=change_index + 1
gold_df['gold_price_change']=gold_price_change
gold_df['gold_price_pct_change']=gold_price_pct_change
stocks_gold_daily_df = pd.merge(djia_df,nasdaq_df, how='outer',on='Date')
stocks_gold_daily_df = stocks_gold_daily_df.merge(sp500_df,how='outer',on='Date')
stocks_gold_daily_df = stocks_gold_daily_df.merge(gold_df,how='outer',on='Date').sort_values(by=['Date'])


In [278]:
#### Export all data to final .CSVs for upload to PostgreSQL
# m1m2_df.to_csv('../DFs_for_DB/m1m2.csv')
# velocity_df.to_csv('../DFs_for_DB/velocity.csv')
# stocks_gold_daily_df.to_csv('../DFs_for_DB/stocks_gold_daily.csv')
# banks_week_month_df.to_csv('../DFs_for_DB/banks_week_month.csv')
# consumer_monthly_df.to_csv('../DFs_for_DB/consumer_monthly.csv')
# consumers_quarterly_df.to_csv('../DFs_for_DB/consumers_quarterly.csv')
# cpi_monthly_df.to_csv('../DFs_for_DB/cpi_monthly.csv')
# federal_reserve_weekly_df.to_csv('../DFs_for_DB/federal_reserve_weekly.csv')
# foreign_trade_month_quarter_df.to_csv('../DFs_for_DB/foreign_trade_month_quarter.csv')
# gdp_quarterly_df.to_csv('../DFs_for_DB/gdp_quarterly.csv')
# government_quarterly_df.to_csv('../DFs_for_DB/government_quarterly.csv')
# investment_month_quarter_df.to_csv('../DFs_for_DB/investment_month_quarter.csv')
# misc_annual_df.to_csv('../DFs_for_DB/misc_annual.csv')
# misc_daily_df.to_csv('../DFs_for_DB/misc_daily.csv')
# ppi_monthly_df.to_csv('../DFs_for_DB/ppi_monthly.csv')